In [ ]:
from yahoo_oauth import OAuth2
import os
import json
import league, game, team
import yhandler
import json
import objectpath
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
# get our session context ("sc")
oauth = OAuth2(None, None, from_file='oauth2.json')

In [ ]:
# prepare functions
def get_json_dump(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return json.dumps(jresp)
def get_json(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return jresp
def get_json_t(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    t = objectpath.Tree(jresp)
    return t

In [ ]:
gm = game.Game(oauth, 'nfl')

#use package to get league_id
league_ids = gm.league_ids()
print(league_ids)
test_league_id = league_ids[2]
print(test_league_id)

# joesleagueid = '390.l.974698'




In [ ]:
###Player RESOURCE
# https://fantasysports.yahooapis.com/fantasy/v2/player/ 

#get a player by player key
playersurl = "league/390.l.974698/players;player_keys=390.p.5479"
player = get_json(playersurl)
player

In [ ]:
#get a player's stats'
statsurl = "league/390.l.974698/players;player_keys=390.p.5479/stats"
stats = get_json(statsurl)
stats

In [ ]:
# players COLLECTION
qburl = "league/390.l.974698/players;position=QB"
qbs = get_json_t(qburl)
qbs

In [ ]:
qbt = get_json_t(qburl)
jfilter = qbt.execute('$..(player)')
qbs = []
for row in jfilter:
#     print(row)
#     print("-----------------------------------------")
#     print("---------------nextrow-------------------")
#     print("-----------------------------------------")
    qbs.append(row)
qbs[0]


In [ ]:
qbs[0]['player'][0]

In [ ]:
# combine into one dictionary
firstqb = qbs[0]['player'][0]
onelist = [item for item in firstqb if item != []]
onelist
onetdict = {k: v for d in onelist for k, v in d.items()}
onetdict

In [ ]:
raw_qb_data = json_normalize(onetdict, sep="_")
raw_qb_data

In [ ]:
all_qb_list = []
allqbs = qbs
allqbs

In [ ]:
# for k, v in allqbs.items():
#     if k != "count": #ignore count item which json adds as autosummary at the end
#         qb = allqbs[k]['player'][0] #go down to individual player level in json
#         qblist = [item for item in qb if item != []]     #remove all empty lists
#         qbdict = { k : v[0] if type(v)==list else v  for d in qblist for k, v in d.items()} #convert all dictionaries into 1 dictionary using comprehension, if they contain a list only use first item from list
#         print(qbdict)
#         all_qb_list.append(qbdict)

for item in allqbs:
#     print(item)
    qb = item['player'][0]
    qblist = [item for item in qb if item != []]     #remove all empty lists
    qbdict = { k : v[0] if type(v)==list else v  for d in qblist for k, v in d.items()} #convert all dictionaries into 1 dictionary using comprehension, if they contain a list only use first item from list
#     print(qbdict)
    all_qb_list.append(qbdict)
print(all_qb_list)
#     if k != "count": #ignore count item which json adds as autosummary at the end
#         qb = allqbs[k]['player'][0] #go down to individual player level in json
#         qblist = [item for item in qb if item != []]     #remove all empty lists
#         qbdict = { k : v[0] if type(v)==list else v  for d in qblist for k, v in d.items()} #convert all dictionaries into 1 dictionary using comprehension, if they contain a list only use first item from list
#         print(qbdict)
#         all_qb_list.append(qbdict)

In [ ]:
qbsdf = json_normalize(all_qb_list, sep="_") #this only gets the sub rows?
qbsdf

In [ ]:
# create universal function for turning API data from a player into a dataframe
positionurl = "league/390.l.974698/players;position=TE"
position_t = get_json_t(positionurl)

position_player_list = []


for item in position_t:
    player = item['player'][0]
    position_player_list = [item for item in player if item != []]     #remove all empty lists
    position_player_dict = { k : v[0] if type(v)==list else v  for d in position_player_list for k, v in d.items()} #convert all dictionaries into 1 dictionary using comprehension, if they contain a list only use first item from list
    position_player_list.append(position_player_dict)
player_position_df = json_normalize(position_player_list, sep="_") #this only gets the sub rows?
player_position_df